In [21]:
with open('19_input.txt') as f:
    block1, block2 = f.read().split('\n\n')
workflows = {}
for line in block1.splitlines():
    name, rules = line[:-1].split('{')
    rules = rules.split(',')
    workflows[name] = ([], rules.pop())
    for rule in rules:
        comp, target = rule.split(':')
        var = comp[0]
        op = comp[1]
        n = int(comp[2:])
        workflows[name][0].append((var, op, n, target))
items = []
for line in block2.splitlines():
    exprs = line[1:-1].split(',')
    item = {}
    for expr in exprs:
        var, val = expr.split('=')
        item[var] = int(val)
    items.append(item)

In [22]:
ops = {'<': int.__lt__, '>': int.__gt__}
def accept(item, name='in'):
    if name == 'R':
        return False
    if name == 'A':
        return True
    rules, fallback = workflows[name]
    for var, op, n, target in rules:
        if ops[op](item[var], n):
            return accept(item, target)
    return accept(item, fallback)

In [23]:
s = 0
for item in items:
    if accept(item):
        s += sum(item.values())
s

487623

In [26]:
def count(ranges, name='in'):
    if name == 'R':
        return 0
    if name == 'A':
        prod = 1
        for lo, hi in ranges.values():
            prod *= hi - lo + 1
        return prod
 
    rules, fallback = workflows[name]
    total = 0
    for var, op, n, target in rules:
        lo, hi = ranges[var]
        if op == '<':
            T = (lo, n - 1)
            F = (n, hi)
        else:
            T = (n + 1, hi)
            F = (lo, n)
        if T[0] <= T[1]:
            cranges = dict(ranges)
            cranges[var] = T
            total += count(cranges, target)
        if F[0] <= F[1]:
            ranges = dict(ranges)
            ranges[var] = F
        else:
            break
    else:
        total += count(ranges, fallback)
    return total

In [27]:
count({var: (1, 4000) for var in 'xmas'})

113550238315130